In [2]:
#import axion_ppa
import priors
import axion_ppa
import numpy as np
import matplotlib.pyplot as plt
from PTMCMCSampler.PTMCMCSampler import PTSampler
from chainconsumer import ChainConsumer
    
PSR_DICT_LIST = axion_ppa.Load_Pulsars()
PSR_NAME_LIST = list(PSR_DICT_LIST.keys())
PSR_DICT_LIST

Do not have mpi4py package.
Do not have acor package


{'J0437-4715': {'PSR': 'J0437-4715',
  'RAJ': '04:37:15.8961737',
  'DECJ': '-47:15:09.110714',
  'DTE_DM': 0.156,
  'PX': 6.37,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': 'Data/J0437-4715_10cm.txt'}},
 'J0613-0200': {'PSR': 'J0613-0200',
  'RAJ': '06:13:43.975901',
  'DECJ': '-02:00:47.23742',
  'DTE_DM': 1.024,
  'PX': 1.01,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': 'Data/J0613-0200_10cm.txt'}},
 'J0614-3329': {'PSR': 'J0614-3329',
  'RAJ': '06:14:10.34823',
  'DECJ': '-33:29:54.1312',
  'DTE_DM': 2.691,
  'PX': 1.1,
  'PX_ERR': 1.1,
  'DATA': {'10cm': 'Data/J0614-3329_10cm.txt'}},
 'J0711-6830': {'PSR': 'J0711-6830',
  'RAJ': '07:11:54.180042',
  'DECJ': '-68:30:47.36454',
  'DTE_DM': 0.106,
  'PX': nan,
  'PX_ERR': nan,
  'DATA': {'10cm': 'Data/J0711-6830_10cm.txt'}},
 'J1017-7156': {'PSR': 'J1017-7156',
  'RAJ': '10:17:51.320769',
  'DECJ': '-71:56:41.61508',
  'DTE_DM': 1.807,
  'PX': 0.6,
  'PX_ERR': 0.6000000000000001,
  'DATA': {'10cm': 'Data/J1017-7156_10cm

In [3]:
# """
# To examine the pulsar distance prior
# """
# psr = axion_ppa.Pulsar(PSR_DICT_LIST[ PSR_NAME_LIST[0]] )
# x = np.linspace(0,2,1000)
# P = [psr.DTE_PRIOR(xi) for xi in x]
# plt.plot(x,np.exp(P))
# plt.grid()
# print(psr.PSR_NAME, psr.DTE)

# lnlike,lnprior,init_gen = psr.gen_spa_likelihood()

In [4]:
#=====================================================#
#    EFAC + EQUAD                                     #
#=====================================================#
for i,psrn in enumerate(PSR_NAME_LIST[:]):  
    psr = axion_ppa.Pulsar( PSR_DICT_LIST[ psrn ] )
    lnlike,lnprior,init_gen = psr.gen_spa_likelihood("10cm")
    
    sampler = PTSampler(2,lnlike,lnprior,cov = np.diag(np.ones(2)),resume=False,outDir="SPA_Chain/"+psr.PSR_NAME,groups=[[0,1]],verbose=False)
    sampler.sample(init_gen(),200000)
    
    chain  = np.loadtxt("SPA_Chain/"+psrn+"/chain_1.txt",skiprows=10000)


    med1 = np.median( chain[:,0])
    min1 = med1 - np.quantile( chain[:,0],0.16 )
    max1 = np.quantile( chain[:,0],0.84 ) - med1


    med2 = np.median( chain[:,1])
    min2 = med2 - np.quantile( chain[:,1],0.16 )
    max2 = np.quantile( chain[:,1],0.84 ) - med2
    
    logVol = np.log( 100*10 )
    logz0 = lnlike([0,-np.inf])
    logBF = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0


    print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR[0])) ,"& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),"&&& %.1f"%(logBF*np.log10(np.exp(1)) )+"\\\\" )

1 & J0437-4715 &-3.63 & $1.67^{+0.03}_{-0.03}$  & $-1.78^{+0.02}_{-0.02}$ &&& 1750391.0\\
2 & J0613-0200 &-1.74 & $0.68^{+0.05}_{-0.06}$  & $-1.01^{+0.04}_{-0.05}$ &&& 4734.4\\
3 & J0614-3329 &-2.00 & $0.79^{+0.05}_{-0.05}$  & $-4.27^{+2.28}_{-2.54}$ &&& 394.3\\
4 & J0711-6830 &-1.70 & $0.74^{+0.02}_{-0.02}$  & $-1.23^{+0.05}_{-0.05}$ &&& 4368.3\\
5 & J1017-7156 &-2.11 & $0.79^{+0.05}_{-0.05}$  & $-1.27^{+0.04}_{-0.04}$ &&& 12283.7\\
6 & J1022+1001 &-2.71 & $0.91^{+0.03}_{-0.04}$  & $-1.89^{+0.05}_{-0.05}$ &&& 10058.6\\
7 & J1024-0719 &-2.36 & $0.89^{+0.03}_{-0.06}$  & $-2.27^{+0.49}_{-3.80}$ &&& 3292.5\\
8 & J1045-4509 &-2.26 & $0.88^{+0.02}_{-0.02}$  & $-4.63^{+2.22}_{-2.33}$ &&& 3264.9\\
9 & J1125-6014 &-2.17 & $0.72^{+0.02}_{-0.02}$  & $-5.02^{+2.08}_{-2.00}$ &&& 1031.6\\
10 & J1545-4550 &-2.65 & $0.73^{+0.06}_{-0.07}$  & $-1.72^{+0.04}_{-0.04}$ &&& 7015.3\\
11 & J1600-3053 &-2.66 & $0.91^{+0.04}_{-0.04}$  & $-1.86^{+0.06}_{-0.08}$ &&& 8215.7\\
12 & J1603-7202 &-1.96 & $0.78^{+0.02

In [5]:
#=====================================================#
#    EFAC only                                        #
#=====================================================#
for i,psrn in enumerate(PSR_NAME_LIST[:]):  
    psr = axion_ppa.Pulsar( PSR_DICT_LIST[ psrn ] )
    lnlike,lnprior,init_gen = psr.gen_spa_likelihood("10cm",p=[-5,5,-8,-7.9])
    
    sampler = PTSampler(2,lnlike,lnprior,cov = np.diag(np.ones(2)),resume=False,outDir="SPA_Chain/m1_"+psr.PSR_NAME,groups=[[0,1]],verbose=False)
    sampler.sample(init_gen(),200000)
    
    chain  = np.loadtxt("SPA_Chain/m1_"+psrn+"/chain_1.txt",skiprows=10000)
    
    med1 = np.median( chain[:,0])
    min1 = med1 - np.quantile( chain[:,0],0.16 )
    max1 = np.quantile( chain[:,0],0.84 ) - med1


    med2 = np.median( chain[:,1])
    min2 = med2 - np.quantile( chain[:,1],0.16 )
    max2 = np.quantile( chain[:,1],0.84 ) - med2
    
    logVol = np.log( 0.1*10 )
    logz0 = lnlike([0,-np.inf])
    logBF = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0


    print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR[0])) ,"& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),"&&& %.1f"%(logBF*np.log10(np.exp(1)) )+"\\\\" )


1 & J0437-4715 &-3.63 & $1.90^{+0.01}_{-0.01}$  & $-7.95^{+0.03}_{-0.03}$ &&& 1750372.2\\
2 & J0613-0200 &-1.74 & $0.91^{+0.02}_{-0.02}$  & $-7.95^{+0.03}_{-0.03}$ &&& 4722.1\\
3 & J0614-3329 &-2.00 & $0.80^{+0.04}_{-0.04}$  & $-7.95^{+0.03}_{-0.03}$ &&& 393.4\\
4 & J0711-6830 &-1.70 & $0.85^{+0.02}_{-0.01}$  & $-7.95^{+0.03}_{-0.03}$ &&& 4355.4\\
5 & J1017-7156 &-2.11 & $1.10^{+0.02}_{-0.02}$  & $-7.95^{+0.03}_{-0.03}$ &&& 12251.0\\
6 & J1022+1001 &-2.71 & $1.05^{+0.02}_{-0.02}$  & $-7.95^{+0.03}_{-0.03}$ &&& 10051.0\\
7 & J1024-0719 &-2.36 & $0.91^{+0.02}_{-0.02}$  & $-7.95^{+0.03}_{-0.03}$ &&& 3290.7\\
8 & J1045-4509 &-2.26 & $0.88^{+0.02}_{-0.02}$  & $-7.95^{+0.03}_{-0.03}$ &&& 3264.2\\
9 & J1125-6014 &-2.17 & $0.72^{+0.02}_{-0.02}$  & $-7.95^{+0.03}_{-0.03}$ &&& 1030.6\\
10 & J1545-4550 &-2.65 & $1.09^{+0.02}_{-0.02}$  & $-7.95^{+0.03}_{-0.03}$ &&& 7003.4\\
11 & J1600-3053 &-2.66 & $1.03^{+0.02}_{-0.02}$  & $-7.95^{+0.03}_{-0.03}$ &&& 8212.0\\
12 & J1603-7202 &-1.96 & $0.91^{+0.02

In [6]:
#=====================================================#
#    EQUAD only                                       #
#=====================================================#
for i,psrn in enumerate(PSR_NAME_LIST[:]):  
    psr = axion_ppa.Pulsar( PSR_DICT_LIST[ psrn ] )
    lnlike,lnprior,init_gen = psr.gen_spa_likelihood("10cm",p=[-0.01,0.01,-8,2])
    
    sampler = PTSampler(2,lnlike,lnprior,cov = np.diag(np.ones(2)),resume=False,outDir="SPA_Chain/m2_"+psr.PSR_NAME,groups=[[0,1]],verbose=False)
    sampler.sample(init_gen(),200000)
    
    chain  = np.loadtxt("SPA_Chain/m2_"+psrn+"/chain_1.txt",skiprows=10000)
    

    med1 = np.median( chain[:,0])
    min1 = med1 - np.quantile( chain[:,0],0.16 )
    max1 = np.quantile( chain[:,0],0.84 ) - med1


    med2 = np.median( chain[:,1])
    min2 = med2 - np.quantile( chain[:,1],0.16 )
    max2 = np.quantile( chain[:,1],0.84 ) - med2
    
    logVol = np.log( 0.02*10 )
    logz0 = lnlike([0,-np.inf])
    logBF = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0


    print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR[0])) ,"& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),"&&& %.1f"%(logBF*np.log10(np.exp(1)) )+"\\\\" )


1 & J0437-4715 &-3.63 & $0.00^{+0.01}_{-0.01}$  & $-1.61^{+0.01}_{-0.01}$ &&& 1750359.9\\
2 & J0613-0200 &-1.74 & $0.00^{+0.01}_{-0.01}$  & $-0.84^{+0.02}_{-0.02}$ &&& 4725.6\\
3 & J0614-3329 &-2.00 & $0.01^{+0.00}_{-0.00}$  & $-1.16^{+0.05}_{-0.04}$ &&& 388.8\\
4 & J0711-6830 &-1.70 & $-0.01^{+0.00}_{-0.00}$  & $-0.81^{+0.02}_{-0.02}$ &&& 4336.5\\
5 & J1017-7156 &-2.11 & $0.00^{+0.01}_{-0.01}$  & $-1.10^{+0.02}_{-0.02}$ &&& 12274.3\\
6 & J1022+1001 &-2.71 & $0.01^{+0.00}_{-0.00}$  & $-1.59^{+0.02}_{-0.02}$ &&& 10038.5\\
7 & J1024-0719 &-2.36 & $0.00^{+0.01}_{-0.01}$  & $-1.41^{+0.02}_{-0.02}$ &&& 3283.7\\
8 & J1045-4509 &-2.26 & $0.00^{+0.01}_{-0.01}$  & $-1.34^{+0.02}_{-0.02}$ &&& 3252.5\\
9 & J1125-6014 &-2.17 & $0.00^{+0.01}_{-0.01}$  & $-1.36^{+0.02}_{-0.02}$ &&& 1012.8\\
10 & J1545-4550 &-2.65 & $0.00^{+0.01}_{-0.01}$  & $-1.59^{+0.02}_{-0.02}$ &&& 7009.3\\
11 & J1600-3053 &-2.66 & $0.00^{+0.01}_{-0.01}$  & $-1.54^{+0.02}_{-0.02}$ &&& 8194.2\\
12 & J1603-7202 &-1.96 & $0.00^{+0.0

In [9]:

#=====================================================#
#    Summarize                                        #
#=====================================================#
test = 0
for i,psrn in enumerate(PSR_NAME_LIST[:]):  
    psr = axion_ppa.Pulsar( PSR_DICT_LIST[ psrn ] )
    lnlike,lnprior,init_gen = psr.gen_spa_likelihood("10cm")
    
    chain  = np.loadtxt("SPA_Chain/"+psrn+"/chain_1.txt",skiprows=10000)
    chain1 = np.loadtxt("SPA_Chain/m1_"+psrn+"/chain_1.txt",skiprows=10000)
    chain2 = np.loadtxt("SPA_Chain/m2_"+psrn+"/chain_1.txt",skiprows=10000)


    med1 = np.median( chain[:,0])
    min1 = med1 - np.quantile( chain[:,0],0.16 )
    max1 = np.quantile( chain[:,0],0.84 ) - med1


    med2 = np.median( chain[:,1])
    min2 = med2 - np.quantile( chain[:,1],0.16 )
    max2 = np.quantile( chain[:,1],0.84 ) - med2
    
    logz0 = lnlike([0,-np.inf])
    test += logz0
    logBF = np.log( 100*10 ) - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0
    logBF1 = np.log( 0.1*10 ) - np.log( np.mean(np.exp( -( chain1[:,-4]-chain1[:,-4].max() ) ) ) )  + chain1[:,-4].max() - logz0
    logBF2 = np.log( 0.02*10 ) - np.log( np.mean(np.exp( -( chain2[:,-4]-chain2[:,-4].max() ) ) ) )  + chain2[:,-4].max() - logz0


    print( i+1,'&',psrn,"&%.2f"%np.log10(np.median(psr.DPA_ERR)) ,
          "& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2),
          "&%.1f&%.1f& %.1f"%(logBF1*np.log10(np.exp(1)),logBF2*np.log10(np.exp(1)),logBF*np.log10(np.exp(1)) )+"\\\\" )

1 & J0437-4715 &-3.63 & $1.67^{+0.03}_{-0.03}$  & $-1.78^{+0.02}_{-0.02}$ &1750372.2&1750359.9& 1750391.0\\
2 & J0613-0200 &-1.74 & $0.68^{+0.05}_{-0.06}$  & $-1.01^{+0.04}_{-0.05}$ &4722.1&4725.6& 4734.4\\
3 & J0614-3329 &-2.00 & $0.79^{+0.05}_{-0.05}$  & $-4.27^{+2.28}_{-2.54}$ &393.4&388.8& 394.3\\
4 & J0711-6830 &-1.70 & $0.74^{+0.02}_{-0.02}$  & $-1.23^{+0.05}_{-0.05}$ &4355.4&4336.5& 4368.3\\
5 & J1017-7156 &-2.11 & $0.79^{+0.05}_{-0.05}$  & $-1.27^{+0.04}_{-0.04}$ &12251.0&12274.3& 12283.7\\
6 & J1022+1001 &-2.71 & $0.91^{+0.03}_{-0.04}$  & $-1.89^{+0.05}_{-0.05}$ &10051.0&10038.5& 10058.6\\
7 & J1024-0719 &-2.36 & $0.89^{+0.03}_{-0.06}$  & $-2.27^{+0.49}_{-3.80}$ &3290.7&3283.7& 3292.5\\
8 & J1045-4509 &-2.26 & $0.88^{+0.02}_{-0.02}$  & $-4.63^{+2.22}_{-2.33}$ &3264.2&3252.5& 3264.9\\
9 & J1125-6014 &-2.17 & $0.72^{+0.02}_{-0.02}$  & $-5.02^{+2.08}_{-2.00}$ &1030.6&1012.8& 1031.6\\
10 & J1545-4550 &-2.65 & $0.73^{+0.06}_{-0.07}$  & $-1.72^{+0.04}_{-0.04}$ &7003.4&7009.3& 7015.3